## NDVI Computation with OpenEO

This notebook demonstrates computing NDVI from Sentinel-2 imagery using the OpenEO API with OIDC authentication.

In [ ]:
import requests
import numpy as np
import rasterio
import matplotlib.pyplot as plt
from IPython.display import Image, display

## Configuration and OIDC Authentication

In [ ]:
# API endpoints
BASE_URL = "https://eoapi.develop.eoepca.org/openeo/"
STAC_URL = "https://eoapi.develop.eoepca.org/stac"
token_endpoint = "https://iam-auth.develop.eoepca.org/realms/eoepca/protocol/openid-connect/token"

# Get OIDC token
response = requests.post(
    token_endpoint,
    headers={"Content-Type": "application/x-www-form-urlencoded"},
    data={
        "grant_type": "password",
        "username": "eric",
        "password": "changeme",
        "client_id": "demo",
        "client_secret": "demo"
    }
)
response.raise_for_status()
access_token = response.json()["access_token"]
auth_header = {"Authorization": f"Bearer oidc/oidc/{access_token}"}

## Get Scene Info from STAC

In [ ]:
item = requests.get(f"{STAC_URL}/collections/sentinel-2-ometepe/items/S2B_16PFT_20250412_0_L2A").json()
bbox = item["bbox"]
preview_url = next((item["assets"][k]["href"] for k in ["thumbnail", "preview", "visual"] if k in item["assets"]), None)

In [ ]:
bbox

## Display Scene Preview

In [ ]:
if preview_url:
    display(Image(url=preview_url))

## Compute NDVI with openEO

This process graph:
1. Loads the Sentinel-2 collection (red and NIR bands)
2. Scales values from 0-10000 to 0-1
3. Subtracts 0.1 offset (surface reflectance adjustment)
4. Clamps values to [0,1] range
5. Computes NDVI = (NIR - Red) / (NIR + Red)
6. Saves result as GeoTIFF

In [ ]:
# Build process graph for NDVI computation
process_graph = {
    "load1": {
        "process_id": "load_collection",
        "arguments": {
            "id": "sentinel-2-ometepe",
            "spatial_extent": {
                "west": bbox[0],
                "south": bbox[1],
                "east": bbox[2],
                "north": bbox[3]
            },
            "temporal_extent": ["2025-04-12T16:20:21Z", "2025-04-12T16:20:22Z"],
            "bands": ["red", "nir"]
        }
    },
    "red_band": {
        "process_id": "array_element",
        "arguments": {
            "data": {"from_node": "load1"},
            "index": 0
        }
    },
    "red_scale": {
        "process_id": "linear_scale_range",
        "arguments": {
            "x": {"from_node": "red_band"},
            "inputMin": 0,
            "inputMax": 10000,
            "outputMin": 0.0,
            "outputMax": 1.0
        }
    },
    "red_reflectance": {
        "process_id": "subtract",
        "arguments": {
            "x": {"from_node": "red_scale"},
            "y": 0.1
        }
    },
    "red_clamped": {
        "process_id": "clip",
        "arguments": {
            "x": {"from_node": "red_reflectance"},
            "min": 0.0,
            "max": 1.0
        }
    },
    "nir_band": {
        "process_id": "array_element",
        "arguments": {
            "data": {"from_node": "load1"},
            "index": 1
        }
    },
    "nir_scale": {
        "process_id": "linear_scale_range",
        "arguments": {
            "x": {"from_node": "nir_band"},
            "inputMin": 0,
            "inputMax": 10000,
            "outputMin": 0.0,
            "outputMax": 1.0
        }
    },
    "nir_reflectance": {
        "process_id": "subtract",
        "arguments": {
            "x": {"from_node": "nir_scale"},
            "y": 0.1
        }
    },
    "nir_clamped": {
        "process_id": "clip",
        "arguments": {
            "x": {"from_node": "nir_reflectance"},
            "min": 0.0,
            "max": 1.0
        }
    },
    "ndvi": {
        "process_id": "normalized_difference",
        "arguments": {
            "x": {"from_node": "nir_clamped"},
            "y": {"from_node": "red_clamped"}
        }
    },
    "save1": {
        "process_id": "save_result",
        "arguments": {
            "data": {"from_node": "ndvi"},
            "format": "GTiff"
        },
        "result": True
    }
}


In [ ]:
# Print process graph for openEO Web Editor
import json
print(json.dumps({"process_graph": process_graph}, indent=2))

## Execute Process Graph and Save Result to File

In [ ]:
result = requests.post(
    f"{BASE_URL}result",
    json={"process": {"process_graph": process_graph}},
    headers=auth_header
)
result.raise_for_status()
with open("ndvi.tif", "wb") as f:
    f.write(result.content)

## Display NDVI Result

In [ ]:
# Read and display the NDVI result
with rasterio.open("ndvi.tif") as src:
    ndvi_data = src.read(1)
    
    # Filter out NaN values for statistics
    valid_data = ndvi_data[~np.isnan(ndvi_data)]
    
    print(f"NDVI Statistics:")
    print(f"  Shape: {ndvi_data.shape}")
    print(f"  Min: {valid_data.min():.3f}, Max: {valid_data.max():.3f}")
    print(f"  Mean: {valid_data.mean():.3f}, Median: {np.median(valid_data):.3f}")
    print(f"  Valid pixels: {len(valid_data):,} / {ndvi_data.size:,}")
    
    # Create visualization
    plt.figure(figsize=(12, 10))
    im = plt.imshow(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.title('NDVI Map (Sentinel-2)', fontsize=16, fontweight='bold')
    plt.axis('off')
    cbar = plt.colorbar(im, label='NDVI', shrink=0.8)
    cbar.set_label('NDVI', rotation=270, labelpad=20)
    plt.tight_layout()
    plt.show()

## NDVI Value Interpretation

- **< 0**: Water, bare soil, or non-vegetated areas
- **0 - 0.2**: Sparse vegetation, desert areas
- **0.2 - 0.4**: Grassland, shrubs, degraded vegetation
- **0.4 - 0.6**: Temperate and tropical forests, crops
- **0.6 - 1.0**: Dense healthy vegetation